# Import Library

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data as data_utils
import numpy as np
import subprocess
import os
import random
import matplotlib.pyplot as plt

from torch.nn.parameter import Parameter
from torch.nn import init
from torch.nn import Module
from torch.nn import init
from torchvision import datasets, transforms
from PIL import Image
from IPython.core.debugger import set_trace


n_tasks = 10
# n_epochs = 4
print_messages = True

In [ ]:
def create_data():
  train_set = datasets.MNIST('./data', train=True, download=True)
  test_set = datasets.MNIST('./data', train=False, download=True)
  X_train = train_set.data.view(60000, 28*28).float()
  y_train = train_set.targets
  X_test = test_set.data.view(10000, 28*28).float()
  y_test = test_set.targets 
  return X_train, y_train, X_test, y_test

import torch
import torch.nn as nn
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as T
from torch import linalg as LA


input_size = 28*28
hidden_size = 64
num_epochs = 4
bs = 256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


X_train, y_train, X_test, y_test = create_data()
# y_train = y_train.view(1, y_train.shape[0])
# y_test = y_test.view(1, y_test.shape[0])
n_train = X_train.shape[0]
# X_train = torch.tensor(X_train, dtype=torch.float)
# y_train = torch.tensor(y_train, dtype=torch.long)
# X_test = torch.tensor(X_test, dtype=torch.float)
# y_test = torch.tensor(y_test, dtype=torch.long)
# print(X_train.shape)
# print(y_train.shape)
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=bs, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=bs, shuffle=False)

class Net(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.last = nn.Linear(hidden_size, 10)
  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.last(out)
    # out = self.relu(out)
    return out
  def u(self,x):
    u = self.forward(x)
    u.requires_grad_(True)
    return u
  
model = Net(input_size, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()    #MSELoss
lr = 0.01
optim = torch.optim.Adam(model.parameters(), lr=lr)

training_loss = []
total_step = len(train_loader)

def train():
  for epoch in range(num_epochs):
    i=0
    for (x,y) in (train_loader):
      i+=1
      x = x.to(device)
      x.requires_grad_(True)
      y = y.to(device)

      output = model(x)
      loss = criterion(output, y)

      optim.zero_grad()
      loss.backward()
      optim.step()

      training_loss.append(loss.item())

      if (i+1) % 10 == 0:
        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
          for x_test,y_test in test_loader:
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            yhat = model(x_test)
            test_loss += criterion(yhat, y_test).item()
            pred = torch.max(yhat.data,1)[1]
            correct += (pred == y_test).sum()
        print ('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.8f}, Test Loss: {:.8f}, Acc: {:.3f}' 
                    .format(epoch+1, num_epochs, i, total_step, loss.item(), test_loss, 100. * correct/len(test_loader.dataset)))
        

X_train.requires_grad_(True)

#1. Before training
L_before = 0
yhat_before = model.u(X_train.to(device))
# print(criterion(y_train.view(1, y_train.shape[0]).to(device), yhat_before))
for i in range(100):
  jac = torch.autograd.functional.jacobian(model.u, (X_train[i].view(-1,28*28)).to(device), create_graph=True)
  l = LA.norm(jac.view(10,28*28), 2)
  L_before = max(L_before, l)
print("Before training, L = ", L_before)

#2. After training
L_after = 0
train()
yhat_after = model.u(X_train.to(device))
# print(criterion(y_train.to(device), yhat_after))
log = 1

for i in range(100):
  jac = torch.autograd.functional.jacobian(model.u, (X_train[i].view(-1,28*28)).to(device), create_graph=True)
  l = LA.norm(jac.view(10,28*28), 2)
  L_after = max(L_after, l)
  if i == log:
    print(jac.view(10,28*28))
print("After training, L = ", L_after)



Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Before training, L =  tensor(0.2526, device='cuda:0', grad_fn=<CopyBackwards>)
Epoch [1/4], Step [9/235], Train Loss: 5.50314951, Test Loss: 164.66328597, Acc: 19.390
Epoch [1/4], Step [19/235], Train Loss: 1.79202485, Test Loss: 73.92517602, Acc: 43.740
Epoch [1/4], Step [29/235], Train Loss: 1.37780190, Test Loss: 54.59385216, Acc: 55.430
Epoch [1/4], Step [39/235], Train Loss: 1.15504384, Test Loss: 45.55958271, Acc: 66.070
Epoch [1/4], Step [49/235], Train Loss: 0.91023302, Test Loss: 37.09065193, Acc: 72.780
Epoch [1/4], Step [59/235], Train Loss: 0.90739602, Test Loss: 33.02949399, Acc: 74.950
Epoch [1/4], Step [69/235], Train Loss: 0.79380929, Test Loss: 30.73508584, Acc: 78.750
Epoch [1/4], Step [79/235], Train Loss: 0.64854139, Test Loss: 27.00860775, Acc: 79.430
Epoch [1/4], Step [89/235], Train Loss: 0.53463775, Test Loss: 25.16336834, Acc: 82.870
Epoch [1/4], Step [99/235], Train Loss: 0.53553331, Test Loss: 23.30355057, Acc: 83.210
Epoch [1/4], Step [109/235], Train Loss: 

# Download package

In [ ]:
pip install ax-platform

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Constructing Data Set

In [ ]:
class ConstructDataset():
  def __init__(self, type_data, data_path, mnist_path):
    self.type_data = type_data
    self.data_path = data_path
    self.mnist_path = mnist_path
    f = np.load(os.path.join(data_path, mnist_path))
    self.x_tr = torch.from_numpy(f["x_train"])
    self.y_tr = torch.from_numpy(f["y_train"]).long()
    self.x_te = torch.from_numpy(f["x_test"])
    self.y_te = torch.from_numpy(f["y_test"]).long()
    f.close()

  def create_one_task(self, type_data, seed, t, n_tasks):   # return one task as np array
    np.random.seed(seed)
    random.seed(seed)

    if type_data == 'rotated_mnist':
      def rotate_dataset(d, rotation):
        result = torch.FloatTensor(d.size(0), 784)  # 60000*784
        tensor = transforms.ToTensor()

        for i in range(d.size(0)):
          img = Image.fromarray(d[i].numpy(), mode="L")
          result[i] = tensor(img.rotate(rotation)).view(784)
        return result     # 60000 vectors-784
      
      min_rot = 1.0 * t / n_tasks * (180.0 - 0.0) + 0.0
      max_rot = 1.0 * (t+1) / n_tasks * (180.0 - 0.0) + 0.0
      rot = random.random() * (max_rot - min_rot) + min_rot
      task_tr = [rotate_dataset(self.x_tr, rot), self.y_tr, rot]
      task_te = [rotate_dataset(self.x_te, rot), self.y_te, rot]

      return task_tr, task_te
    
    if type_data == 'permuted_mnist':
      def permute(d, permutation):   # d[i] is flattened
        res = torch.FloatTensor(d.shape[0], 784)
        for i in range(d.shape[0]):
          res[i] = d[i][permutation]
        return res

      permutation = np.random.permutation(784)
      task_tr = [permute(self.x_tr.view(self.x_tr.shape[0], 28*28), permutation), self.y_tr]
      task_te = [permute(self.x_te.view(self.x_te.shape[0], 28*28), permutation), self.y_te]

      return task_tr, task_te

  def create_several_tasks(self, num_tasks, file_name, type_data):
    tasks_tr = []
    tasks_te = []
    if file_name != "":   # if save constructed data to file
      path = file_name + "_" + str(num_tasks) + ".pt"
      if not os.path.exists(os.path.join(self.data_path, path)):
        for t in range(num_tasks):
          task_tr, task_te = self.create_one_task(type_data, t, t, num_tasks) 
          tasks_tr.append(task_tr)
          tasks_te.append(task_te)
        torch.save([tasks_tr, tasks_te], self.data_path + "/" + path)
      return self.data_path + "/" + path
    else:   # load all task to RAM
      if type_data == 'rotated_mnist':
        path = "mnist_rotations" + "_" + str(num_tasks) + ".pt"
        if not os.path.exists(os.path.join(self.data_path, path)):
          for t in range(num_tasks):
            task_tr, task_te = self.create_one_task(type_data, t, t, num_tasks) 
            tasks_tr.append(task_tr)
            tasks_te.append(task_te)
          # torch.save([tasks_tr, tasks_te], self.data_path + "/" + path)
        else:
          tasks_tr, tasks_te = torch.load(self.data_path + "/" + path)
      if type_data == 'permuted_mnist':
        path = "mnist_permutations" + "_" + str(num_tasks) + ".pt"
        if not os.path.exists(os.path.join(self.data_path, path)):
          for t in range(num_tasks):
            task_tr, task_te = self.create_one_task(type_data, t, t, num_tasks) 
            tasks_tr.append(task_tr)
            tasks_te.append(task_te)
          # torch.save([tasks_tr, tasks_te], self.data_path + "/" + path)
        else:
          tasks_tr, tasks_te = torch.load(self.data_path + "/" + path)
      return tasks_tr, tasks_te

In [ ]:
data_path = '/content/drive/MyDrive/Colab_Notebooks/LAB/Some Models/SI/data'
data = ConstructDataset('permuted_mnist', data_path, 'mnist.npz')
tasks_tr, tasks_te = data.create_several_tasks(n_tasks, "", 'permuted_mnist')

# Defining Synaptic Intelligence Model (Simple)

In [ ]:
class SIModel(nn.Module):
    def __init__(self):
        super(SIModel, self).__init__()

        # SI Hyperparameters
        self.si_c = 0.          # regularization strength
        self.epsilon = 0.1      # dampening parameter
    
    def init_weights(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.kaiming_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def init(self, n_neurons):
        # Simple MLP
        self.net = nn.Sequential(
            nn.Linear(28*28, n_neurons),
            nn.ReLU(),
            nn.Linear(n_neurons, n_neurons),
            nn.ReLU(),
            nn.Linear(n_neurons, n_neurons),
            nn.ReLU(),
            nn.Linear(n_neurons, 10)
        )
        self.net.apply(self.init_weights)

    def forward(self, x):
        return self.net(x)

    def update_omega(self, W):
        '''
        After completing training on a task, update the per-parameter regularization strength.
        [W] <dict> estimated parameter-specific contribution to changes in total loss of completed task
        '''

        # Loop over all parameters
        for n, p in self.named_parameters():
            if p.requires_grad:
                n = n.replace('.', '__')

                # Find/calculate new values for quadratic penalty on parameters
                p_prev = getattr(self, '{}_SI_prev_task'.format(n))
                p_current = p.detach().clone()
                p_change = p_current - p_prev
                
                omega_add = W[n]/(p_change**2 + self.epsilon)
                try:
                    omega = getattr(self, '{}_SI_omega'.format(n))
                except AttributeError:
                    omega = p.detach().clone().zero_()
                omega_new = omega + omega_add

                # Store these new values in the model
                self.register_buffer('{}_SI_prev_task'.format(n), p_current)
                self.register_buffer('{}_SI_omega'.format(n), omega_new)

    def surrogate_loss(self):
        '''
        Calculate SI's surrogate loss.
        '''
        try:
            losses = []
            for n, p in self.named_parameters():
                if p.requires_grad:
                    # Retrieve previous parameter values and their normalized path integral (i.e., omega)
                    n = n.replace('.', '__')
                    prev_values = getattr(self, '{}_SI_prev_task'.format(n))
                    omega = getattr(self, '{}_SI_omega'.format(n))
                    # Calculate SI's surrogate loss, sum over all parameters
                    losses.append((omega * (p-prev_values)**2).sum())
            return sum(losses)
        except AttributeError:
            # SI-loss is 0 if there is no stored omega yet
            return torch.tensor(0., device=self._device())

    def _device(self):
        return next(self.parameters()).device


# Running experiment

In [ ]:
def train_task(model, device, train_loader, optimizer, batch_log, n_epochs):

    model.train()

    # Prepare <dicts> to store running importance estimates and param-values before update
    W = {}
    param_old = {}
    for name, param in model.named_parameters():
        if param.requires_grad:
            name = name.replace('.', '__')
            W[name] = param.data.clone().zero_()
            param_old[name] = param.data.clone()

    losses = []
    total_losses = []
    for k in range(n_epochs):
        if print_messages:
            print("----> Epoch {}:".format(k))
        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            # Get the prediction
            y_hat = model(x)
            # Calculate training-precision
            precision = (y == y_hat.max(1)[1]).sum().item() / x.size(0)
            # Calculate the loss using CROSS ENTROPY and the SURROGATE LOSS
            loss = F.cross_entropy(input=y_hat, target=y, reduction='mean')
            surrogate_loss = model.surrogate_loss()
            total_loss = loss + model.si_c * surrogate_loss
            # Backpropagate errors
            total_loss.backward()
            # Take optimization-step
            optimizer.step()

            # Update running parameter importance estimates in W
            # "In practice, we can approximate w as the running sum of the 
            # product of the gradient g(w) and the parameter update" 
            for name, param in model.named_parameters():
                if param.requires_grad:
                    name = name.replace('.', '__')
                    if param.grad is not None:
                        W[name].add_(-param.grad*(param.detach()-param_old[name]))
                    param_old[name] = param.detach().clone()

            # Print out a log
            if batch_idx % batch_log == 0:
                losses.append(loss.item())
                total_losses.append(total_loss.item())
                if print_messages:
                    print('---->[{}/{} ({:.0f}%)]\tPrecision: {:.6f}\tLoss: {:.6f}\tSurrogate Loss: {:.6f}\tTotal Loss: {:.6f}'.format(
                        batch_idx * len(x), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), 
                        precision, loss.item(), surrogate_loss.item(), total_loss.item()))
            
    # After finishing training on a task, update the omega value in the model
    model.update_omega(W)

    return losses, total_losses
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            test_loss += F.cross_entropy(input=y_hat, target=y, reduction='mean')
            pred = y_hat.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    return correct, test_loss

def eval_on_tasks(model, device, test_loaders):
    acc = []  # accuracy of each task
    test_losses = []     # loss of each task
    for j in range(n_tasks):
        correct, test_loss = test(model, device, test_loaders[j])
        acc.append(correct / len(test_loaders[j].dataset))
        test_losses.append(test_loss)
        if print_messages:
            print('---->Test set {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
                j, test_loss, correct, len(test_loaders[j].dataset),
                100. * correct / len(test_loaders[j].dataset)))
    return acc, test_losses

In [ ]:
config={
    "lr": 0.003, 
    "si_c": 0.1, 
    "si_epsilon": 0.01,
    "optimizer": "adam",
    "batch_size": 64,
    "n_neurons": 2000,
    "sample_size": 60000
    }

In [ ]:
def main(config, n_epochs): 
    # Use cuda?
    cuda = torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")

    # Create Model
    model = SIModel()
    model.init(config["n_neurons"])
    model.to(device)
    optim_list = [{'params': filter(lambda p: p.requires_grad, model.parameters()), 'lr': config['lr']}]
    if config['optimizer'] == "adam":
        optimizer = optim.Adam(optim_list, betas=(0.9, 0.999))
    else:
        optimizer = optim.SGD(optim_list)

    # SI Parameters
    model.si_c = config["si_c"]
    model.epsilon = config["si_epsilon"]

    for name, param in model.named_parameters():
        if param.requires_grad:
            name = name.replace('.', '__')
            model.register_buffer('{}_SI_prev_task'.format(name), param.data.clone())

    # Load test data
    test_loaders = []
    for i in range(n_tasks):
        test_loaders.append(data_utils.DataLoader(data_utils.TensorDataset(tasks_te[i][0], tasks_te[i][1]), batch_size=1000, shuffle = False))

    # Training
    if print_messages:
        print("--> Training:")

    total_acc = []
    total_test_losses = []
    
    # Before we start training we will get a baseline by evaluating our tasks
    acc, test_losses = eval_on_tasks(model, device, test_loaders)
    total_acc.append(acc)
    total_test_losses.append(test_losses)

    for i in range(n_tasks):
        if print_messages:
            print("--> Training Task {}:".format(i))

        perm = np.random.permutation(tasks_tr[i][1].size(0))    # data shuffle
        perm = perm[:config['sample_size']]
        train_data = data_utils.TensorDataset(tasks_tr[i][0], tasks_tr[i][1])
        train_loader = data_utils.DataLoader(train_data, batch_size=config["batch_size"], 
                                      sampler = data_utils.SubsetRandomSampler(perm), drop_last = True)
        
        train_losses, total_train_losses = train_task(model, device, train_loader, optimizer, 200, n_epochs)
        
        # Reset the optimizer (if using adam)
        if config['optimizer'] == "adam":
            model.optimizer = optim.Adam(optim_list, betas=(0.9, 0.999))

        if print_messages:
            print(train_losses)
            print(total_train_losses)
            print("--> Finished Training Task {}. Starting Test phase:".format(i))

        acc, test_losses = eval_on_tasks(model, device, test_loaders)
        total_acc.append(acc)
        total_test_losses.append(test_losses)
    
    haha=model

    return total_acc, total_test_losses

In [ ]:
# result_path = '/content/drive/MyDrive/Colab_Notebooks/LAB/Some Models/SI/result/over_parameter'

# n_epochs_range = [1,2,3,4,5,6,7,8,9,10]

# for n_epochs in n_epochs_range:
#   total_acc, total_test_losses = main(config, n_epochs)    # acc of 10 tasks after training each task 

#   # Get the accuracy metric as defined by Facebook paper: sum(R_Ti) 
#   # where T is the test set of the last Task and i is the current trained task
#   average_acc = np.mean(total_acc[n_tasks])
#   print("Accuracy:", average_acc)
#   print("Confusion matrix:")
#   print('\n'.join([','.join([str(item) for item in row]) for row in total_acc]))

#   result_name = '0.1_e' + str(n_epochs) + '_big_si_permuted_10'
#   with open(result_path + "/" + result_name + ".txt", "w") as f:
#     for t in range(1, n_tasks+1):
#       f.write("{} : ".format(str(t)))
#       for acc in total_acc[t]:
#         f.write("{:.4f} ".format(acc))
#       # f.write(" ".join(str(total_acc[t])))
#       f.write("\n")

# only performance on task 1 is good????

--> Training:
---->Test set 0: Average loss: 0.1601, Accuracy: 774/10000 (8%)
---->Test set 1: Average loss: 0.1545, Accuracy: 882/10000 (9%)
---->Test set 2: Average loss: 0.1657, Accuracy: 878/10000 (9%)
---->Test set 3: Average loss: 0.1313, Accuracy: 1173/10000 (12%)
---->Test set 4: Average loss: 0.1961, Accuracy: 830/10000 (8%)
---->Test set 5: Average loss: 0.1488, Accuracy: 1084/10000 (11%)
---->Test set 6: Average loss: 0.1650, Accuracy: 1070/10000 (11%)
---->Test set 7: Average loss: 0.1542, Accuracy: 1341/10000 (13%)
---->Test set 8: Average loss: 0.1617, Accuracy: 669/10000 (7%)
---->Test set 9: Average loss: 0.1588, Accuracy: 656/10000 (7%)
--> Training Task 0:
----> Epoch 0:
---->[0/60000 (0%)]	Precision: 0.046875	Loss: 169.323257	Surrogate Loss: 0.000000	Total Loss: 169.323257
---->[12800/60000 (21%)]	Precision: 0.734375	Loss: 0.937317	Surrogate Loss: 0.000000	Total Loss: 0.937317
---->[25600/60000 (43%)]	Precision: 0.906250	Loss: 0.624988	Surrogate Loss: 0.000000	Total 

KeyboardInterrupt: ignored

In [ ]:
result_path = '/content/drive/MyDrive/Colab_Notebooks/LAB/Some Models/SI/result/over_parameter'

n_epochs_range = [6] #,7,8,9,10]

haha = None

for n_epochs in n_epochs_range:
  total_acc, total_test_losses = main(config, n_epochs)    # acc of 10 tasks after training each task 

  # Get the accuracy metric as defined by Facebook paper: sum(R_Ti) 
  # where T is the test set of the last Task and i is the current trained task
  average_acc = np.mean(total_acc[n_tasks])
  print("Accuracy:", average_acc)
  print("Confusion matrix:")
  print('\n'.join([','.join([str(item) for item in row]) for row in total_acc]))

  result_name = '0.1_e' + str(n_epochs) + '_big_si_permuted_10'
  with open(result_path + "/" + result_name + ".txt", "w") as f:
    for t in range(1, n_tasks+1):
      f.write("{} : ".format(str(t)))
      for acc in total_acc[t]:
        f.write("{:.4f} ".format(acc))
      # f.write(" ".join(str(total_acc[t])))
      f.write("\n")



--> Training:
---->Test set 0: Average loss: 0.1767, Accuracy: 1225/10000 (12%)
---->Test set 1: Average loss: 0.2056, Accuracy: 1172/10000 (12%)
---->Test set 2: Average loss: 0.1637, Accuracy: 1097/10000 (11%)
---->Test set 3: Average loss: 0.1905, Accuracy: 1092/10000 (11%)
---->Test set 4: Average loss: 0.1548, Accuracy: 893/10000 (9%)
---->Test set 5: Average loss: 0.1720, Accuracy: 704/10000 (7%)
---->Test set 6: Average loss: 0.2250, Accuracy: 844/10000 (8%)
---->Test set 7: Average loss: 0.1697, Accuracy: 1013/10000 (10%)
---->Test set 8: Average loss: 0.1630, Accuracy: 832/10000 (8%)
---->Test set 9: Average loss: 0.2288, Accuracy: 959/10000 (10%)
--> Training Task 0:
----> Epoch 0:
---->[0/60000 (0%)]	Precision: 0.140625	Loss: 164.804474	Surrogate Loss: 0.000000	Total Loss: 164.804474
---->[12800/60000 (21%)]	Precision: 0.843750	Loss: 0.792060	Surrogate Loss: 0.000000	Total Loss: 0.792060
---->[25600/60000 (43%)]	Precision: 0.906250	Loss: 0.282400	Surrogate Loss: 0.000000	Tot

In [ ]:
result_path = '/content/drive/MyDrive/Colab_Notebooks/LAB/Some Models/SI/result/over_parameter'
# 10 tasks permuted:
# 1. c=0, hidden_neurons = 100
# 2. c = [0.1, 0.01, 1, 0.001]
result_name = '0.1_e6_small_si_permuted_10'
with open(result_path + "/" + result_name + ".txt", "w") as f:
  for t in range(1, n_tasks+1):
    f.write("{} : ".format(str(t)))
    for acc in total_acc[t]:
      f.write("{:.4f} ".format(acc))
    # f.write(" ".join(str(total_acc[t])))
    f.write("\n")

In [ ]:
print(n_epochs)
print(total_acc)


8
[[0.0882, 0.1071, 0.0915, 0.118, 0.1085, 0.0606, 0.0731, 0.0982, 0.073, 0.1115], [0.9569, 0.1415, 0.1122, 0.0993, 0.1068, 0.1461, 0.0639, 0.0834, 0.1037, 0.0927], [0.9074, 0.0958, 0.0956, 0.098, 0.0954, 0.107, 0.0741, 0.1129, 0.1022, 0.0691], [0.9082, 0.0958, 0.1032, 0.1098, 0.1027, 0.1032, 0.1033, 0.104, 0.1055, 0.102], [0.9107, 0.0957, 0.1032, 0.1032, 0.1031, 0.1033, 0.1032, 0.1032, 0.1032, 0.1032], [0.9108, 0.0957, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032], [0.9107, 0.0957, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032], [0.9113, 0.0962, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032], [0.9112, 0.0959, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032], [0.9114, 0.0957, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032], [0.9114, 0.0955, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032, 0.1032]]


# Tune Hyperparamters using Ax

In [ ]:
def tune(config, objective):
    total_acc, total_loss = main(config)
    if objective == "accuracy":
        return np.mean(total_acc[n_tasks-1])
    elif objective == "loss":
        return np.mean(total_loss[n_tasks-1])
    else:
        return

from ax import optimize
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "lr",
            "type": "range",
            "bounds": [1e-4, 0.4], 
            "log_scale": True,
            "value_type": "float",
        },
        {  
            "name": "si_c",
            "type": "range",
            "bounds": [0.01, 0.5],
            "value_type": "float",
        },
        {  
            "name": "si_epsilon",
            "type": "fixed",
            "value": 0.01,
            "value_type": "float",
        },
        {  
            "name": "batch_size",
            "type": "fixed",
            "value": 64,
            "value_type": "int",
        },
        {  
            "name": "sample_size",
            "type": "fixed",
            "value": 10000,
            "value_type": "int",
        },
        {  
            "name": "n_neurons",
            "type": "fixed",
            "value": 100,
            "value_type": "int",
        },
        {  
            "name": "optimizer",
            "type": "fixed",
            "value": "adam",
            "value_type": "str",
        },
    ],
    evaluation_function=lambda p: tune(p, "accuracy"),
    objective_name='accuracy',
)
print(best_parameters)
print(values)
    #evaluation_function=lambda p: np.mean(main(p)[n_tasks-1]),
    #minimize=True,)

[INFO 04-05 20:41:10] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 7 arms, GPEI for subsequent arms], generated 0 arm(s) so far). Iterations after 7 will take longer to generate due to model-fitting.
[INFO 04-05 20:41:10] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 04-05 20:41:10] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-05 20:42:21] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-05 20:43:32] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-05 20:44:44] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-05 20:45:55] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-05 20:47:06] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-05 20:48:17] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-05 20:49:28] ax.service.managed_loop: Running optimization t

{'lr': 0.002969074196382516, 'si_c': 0.441531161522652, 'si_epsilon': 0.01, 'batch_size': 64, 'sample_size': 10000, 'n_neurons': 100, 'optimizer': 'adam'}
({'accuracy': 0.6400499142783531}, {'accuracy': {'accuracy': 1.1108064544778255e-09}})


In [ ]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "lr",
            "type": "range",
            "bounds": [1e-4, 0.4], 
            "log_scale": True,
            "value_type": "float",
        },
        {  
            "name": "si_c",
            "type": "fixed",
            "value": 0.152,
            "value_type": "float",
        },
        {  
            "name": "si_epsilon",
            "type": "fixed",
            "value": 0.01,
            "value_type": "float",
        },
        {  
            "name": "batch_size",
            "type": "choice",
            "values": [64, 128, 256],
            "value_type": "int",
        },
        {  
            "name": "sample_size",
            "type": "choice",
            "values": [1000, 5000, 10000, 20000, 40000, 60000],
            "value_type": "int",
        },
        {  
            "name": "n_neurons",
            "type": "fixed",
            "value": 100,
            "value_type": "int",
        },
        {  
            "name": "optimizer",
            "type": "fixed",
            "value": "adam",
            "value_type": "str",
        },
    ],
    evaluation_function=lambda p: tune(p, "accuracy"),
    objective_name='accuracy',
)
print(best_parameters)
print(values)

[INFO 04-05 21:05:27] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-05 21:05:27] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 04-05 21:05:27] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-05 21:06:12] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-05 21:06:51] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-05 21:08:05] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-05 21:08:57] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-05 21:11:03] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-05 21:13:53] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-05 21:14:45] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-05 21:15:29] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-05 21:17:34] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-05 21:20:38] ax.service.ma

{'lr': 0.004312738958492966, 'batch_size': 256, 'sample_size': 60000, 'si_c': 0.152, 'si_epsilon': 0.01, 'n_neurons': 100, 'optimizer': 'adam'}
({'accuracy': 0.551365}, {'accuracy': {'accuracy': 0.0}})
